# Final Project Part - II

In this part, we will be implementing the tables and loading data into the tables that we have designed in Part - I. 

## 3.1 Using DDL, create each of the relations in the postgres server. 

* Use `dsa_student` database
* You are free to use any of the following tools:
  * psql
      * If you use psql, copy and paste your query in the following cell
  * sql magic
  * psycopg2
  * SQLAlchemy
* Add additional cells if required

In [1]:
import getpass
import pandas as pd
import psycopg2
import sqlalchemy
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine

In [2]:
mypasswd = getpass.getpass()
username = 'jpsvf2'
host = 'pgsql.dsa.lan'
database = 'dsa_student'

postgres_db = {'drivername': 'postgres',
               'username': username,
               'password': mypasswd,
               'host': host,
               'database' :database}
engine = create_engine(URL(**postgres_db), echo=False)
del mypasswd

········


In [3]:
query = """
DROP TABLE IF EXISTS jpsvf2.iucr CASCADE;
CREATE TABLE jpsvf2.iucr (
    IUCR VARCHAR(50),
    Primary_Type VARCHAR(50),
    Description VARCHAR(100),
    CONSTRAINT pk_iucr
        PRIMARY KEY (IUCR)
);

DROP TABLE IF EXISTS jpsvf2.location_info CASCADE;
CREATE TABLE jpsvf2.location_info (
    Location_ID INT,
    Block VARCHAR(200),
    Beat VARCHAR(200),
    District VARCHAR(200),
    Location_Description VARCHAR(300),
    Community_Area VARCHAR(200),
    Ward VARCHAR(100),
    X_Coordinate REAL,
    Y_Coordinate REAL,
    Latitude REAL,
    Longitude REAL,
    CONSTRAINT pk_location_info
        PRIMARY KEY (Location_ID)
);

DROP TABLE IF EXISTS jpsvf2.case_info CASCADE;
CREATE TABLE jpsvf2.case_info (
    Case_Number VARCHAR(50),
    Arrest BOOL,
    FBI_Code VARCHAR(50),
    Domestic BOOL,
    IUCR VARCHAR(50),
    Date TIMESTAMP,
    Location_ID INT,
    CONSTRAINT pk_case_info
        PRIMARY KEY (Case_Number),
    CONSTRAINT fk1_case_info
        FOREIGN KEY (IUCR)
            REFERENCES jpsvf2.iucr(IUCR),
    CONSTRAINT fk2_case_info
        FOREIGN KEY (Location_ID)
            REFERENCES jpsvf2.location_info(Location_ID)
);

DROP TABLE IF EXISTS jpsvf2.record CASCADE;
CREATE TABLE jpsvf2.record (
    ID INT,
    Case_Number VARCHAR(50),
    Updated_On TIMESTAMP,
    CONSTRAINT pk_record
        PRIMARY KEY (ID),
    CONSTRAINT fk_record
        FOREIGN KEY (Case_Number)
            REFERENCES jpsvf2.case_info(Case_Number)
);
"""

with engine.connect() as connection:
    res = connection.execute(query)
    print(res)


## 3.2 Show the table definitions using psql or querying information_schema.colums catalog
* Add additional cells if required

## 4.1 Load the data from the given csv file to the relations



* Assuming there will be more than one relations, you need to extract a subsets of data from the csv data. As Python may not be your first choice, you can use any languages to create subsets of data. Then store these data into the M8 exercises folder. 
* After curating the data use any of the following tools to load the data into the tables
  * psql
      * If you use psql, copy and paste your command/query in the following cell
  * sql magic
  * psycopg2
  * SQLAlchemy
* Add additional cells if required

In [4]:
iucr_file = 'iucr.csv'

iucr = pd.read_csv(iucr_file)

iucr.to_sql('iucr', 
          engine,             
          schema= username,   
          if_exists='append',
          index=False,        
          chunksize=300)  

with engine.connect() as connection:
    res = connection.execute("select * from jpsvf2.iucr limit 2")
    for row in res:
        print(row)

('1562', 'SEX OFFENSE', 'AGG CRIMINAL SEXUAL ABUSE')
('1544', 'SEX OFFENSE', 'SEXUAL EXPLOITATION OF A CHILD')


In [5]:
location_info_file = 'location_info.csv'

location_info = pd.read_csv(location_info_file)

location_info.to_sql('location_info', 
          engine,             
          schema= username,   
          if_exists='append',
          index=False,        
          chunksize=300)  

with engine.connect() as connection:
    res = connection.execute("select * from jpsvf2.location_info limit 2")
    for row in res:
        print(row)

(92623, '026XX N MC VICKER AVE', '2512', '25', 'RESIDENCE', '19.0', '29.0', None, None, None, None)
(120498, '036XX S RHODES AVE', '212', '2', 'APARTMENT', '35.0', '4.0', None, None, None, None)


In [6]:
case_info_file = 'case_info.csv'

case_info = pd.read_csv(case_info_file)

case_info.to_sql('case_info', 
          engine,             
          schema= username,   
          if_exists='append',
          index=False,        
          chunksize=300)  

with engine.connect() as connection:
    res = connection.execute("select * from jpsvf2.case_info limit 2")
    for row in res:
        print(row)

('HZ170962', True, '17', False, '1562', datetime.datetime(2012, 1, 1, 0, 0), 92623)
('HZ170983', True, '17', False, '1544', datetime.datetime(2012, 1, 1, 0, 0), 92623)


In [7]:
record_file = 'record.csv'

record = pd.read_csv(record_file)

record.to_sql('record', 
          engine,             
          schema= username,   
          if_exists='append',
          index=False,        
          chunksize=300)  

with engine.connect() as connection:
    res = connection.execute("select * from jpsvf2.record limit 2")
    for row in res:
        print(row)

(10433096, 'HZ170962', datetime.datetime(2016, 5, 11, 15, 48))
(10433124, 'HZ170983', datetime.datetime(2016, 5, 11, 15, 48))


## 4.2 For each of the tables, show the number of rows in the table using a sql query

* Add additional cells if required

In [8]:
with engine.connect() as connection:
    df2 = pd.read_sql_query("SELECT COUNT(*) FROM iucr", connection)

df2.head()

,count
0,318


In [9]:
with engine.connect() as connection:
    df3 = pd.read_sql_query("SELECT COUNT(*) FROM location_info", connection)

df3.head()

,count
0,243427


In [10]:
with engine.connect() as connection:
    df4 = pd.read_sql_query("SELECT COUNT(*) FROM case_info", connection)

df4.head()

,count
0,334715


In [11]:
with engine.connect() as connection:
    df5 = pd.read_sql_query("SELECT COUNT(*) FROM record", connection)

df5.head()

,count
0,334715
